## 理論1層: 熱による進化

物語: 鍛冶屋は鉄を高温で熱しては、金槌ちで叩き、水で冷やし、繰り返し、良い鉄に仕上げる

なぜ：最適化（進化に似てる）が成長や教育のような最重要の概念であり、説明が最も簡単である

特徴: 大域進化、機械評価、自動探索、汎用

注意: 多くの時間を消費する

## シミュレーション

### 焼きなまし法

#### 計算手順

- 計算できる正しさがある。(評価関数)
- 高い温度がある。
- 無作為の状態から始まる。
- a. 今の状態の近くに次の状態がある。近くから選ぶ。(近傍)
- 1. 次の状態が良い状態なら変化する。
- 2. 高い温度なら高確率で変化する。低い温度なら低確率で変化する。
- 熱は下がる。
- a.から繰り返す。
- 最終的に評価の高い変化しにくい状態となる。

### 巡回セールスマン問題で焼き鈍し法を体験

- 物語：販売員が営業先の拠点を全て回るが、楽をしたい。
- 理論的モデル：各点を最小の距離になるように結ぶ。
- 評価関数：営業先の拠点の訪問順による総距離。
- 近く（近傍）：営業先の拠点の訪問順を交換する。
    -  2番目, 1番目に訪れるなら[3,2,1]で、 1,2で置き換え [2,3,1] 1,3で置き換え [1,2,3], 2,3 で置き換え [3,1,2]となります。 
	- [1,3,2] [2, 1 ,3] は　近くありません。

### 手順

- 上の理論から想像しましょう。どうなるでしょうか。
    - 下にプログラムがありますので詳細を確認してもよいかもしれません。
- 上部メニュー ＞ Kernel ＞ Restart ＆ Run All を押す
- 画像を比較する。総距離を比較する。考察する。
- 上記を繰り返す。
- 興味があれば、パラメータ・プログラムを改変して、手順を再度行います。


In [ ]:
# 言語 python
# グローバル変数を使う癖は止めた方がいいかもしれないです。
# 比較的わかりやすく書きました。

import random
import numpy as np
import math
from PIL import Image, ImageDraw

DRAW_TIMES = 10 # 図を描く回数
DRAW_ITER = 10000 # 図を描くまでのループ回数
MAX_ITER = DRAW_TIMES * DRAW_ITER
WIDTH = 300  # 図の幅
HEIGHT = 300 # 図の高さ
TEMP_DECAY_FACTOR = 0.15 # 温度の基準
COUNT = 9 # 点の数

global min_total_dist # 比較用最小距離：初期値は大きければいい。
min_total_dist = 1000000
global min_order_vertex # 最適状態記録
min_order_vertex = []
global vertex_pos # 訪問点の位置 array[[x, y]]
vertex_pos = []
global order_vertex # 状態:頂点の順番
order_vertex = None
global iteration # 繰り返し回数
iteration = 0
global temp # 温度
temp = 0
global dist_mat # 距離行列 array([[]])
dist_mat = [[]]
global total_dist # 総距離：目的・評価
total_dist = 0

def drawLine(drawObj, px, py, ax, ay):
    drawObj.line([(int(px* WIDTH), int(py*HEIGHT)), (int(ax*WIDTH), int(ay*HEIGHT))], fill="blue", width=1)

def makeArray2D(arr):
    ret = []
    for i in range(len(arr)):
        ret.append(list(range(len(arr))))
        for j in range(len(arr)):
            ret[i][j] = 0
    return np.asarray(ret, dtype=np.float64)

def twice(x):
    return x * x

def shuffle(arr):
    for i in range(0, len(arr)-1):
        j = 0 | random.randint(0, i + 1)
        swap = arr[i]
        arr[i] = arr[j]
        arr[j] = swap
    return arr

def drawPath(arg_order_vertex):
    im = Image.new('RGB', (WIDTH, HEIGHT), (192, 192, 192))
    drawObj = ImageDraw.Draw(im)

    for i in range(1, COUNT):
        drawLine(drawObj, vertex_pos[arg_order_vertex[i-1]][0], vertex_pos[arg_order_vertex[i-1]][1], vertex_pos[arg_order_vertex[i]][0],vertex_pos[arg_order_vertex[i]][1])
    drawLine(drawObj,vertex_pos[arg_order_vertex[COUNT-1]][0], vertex_pos[arg_order_vertex[COUNT-1]][1], vertex_pos[arg_order_vertex[0]][0],vertex_pos[arg_order_vertex[0]][1])
    im.show()

def draw():
    total_dist = calclateTotalDistance(order_vertex)
    print("ーーーーーーーーーーーーー")
    print("繰返回数 : "+ str(iteration))
    print("温度 : "+ str(temp))
    print("総距離 : "+  str(total_dist))
    print("最小距離 : "+ str(min_total_dist))
    drawPath(order_vertex)

def sa(n, m):
    for i in range(n):
        for j in range(m):
            simulatedAnealing()
        draw()

def simulatedAnealing():
    global temp
    global iteration
    global total_dist
    global min_total_dist
    global min_order_vertex
    global order_vertex
    temp = temperature()
    order_vertex = calclateSA(order_vertex)
    total_dist = calclateTotalDistance(order_vertex)
    if (total_dist < min_total_dist):
        min_total_dist = total_dist
        min_order_vertex = order_vertex
    iteration += 1

def setupSA():
    global order_vertex
    global vertex_pos
    global dist_mat
    global min_order_vertex
    global min_total_dist

    for i in range(COUNT):
        vertex_pos.append([0, 0])
        vertex_pos[i][0] = random.random()
        vertex_pos[i][1] = random.random()
    vertex_pos = np.asarray(vertex_pos, dtype=np.float64)
    order_vertex = np.asarray(shuffle(list(range(COUNT))))
    dist_mat = makeArray2D(order_vertex)
    calcDistanceCache()
    min_order_vertex = list(range(COUNT))
    min_total_dist = 100000
    drawPath(order_vertex)
    
# 距離行列の計算
def calcDistanceCache():
    global dist_mat
    for i in range(COUNT):
        for j in range(COUNT):
            if(i == j):
                dist_mat[i][j]=0
            else:
                dist_mat[i][j] = math.sqrt(twice(vertex_pos[i][0] - vertex_pos[j][0]) + twice(vertex_pos[i][1] - vertex_pos[j][1]))

#　温度
def temperature():
    return TEMP_DECAY_FACTOR ** (iteration / MAX_ITER)

# 2opt法の計算
def calclateSA(arg_order_vertex):
    for i in range(0, COUNT - 1):
        for j in range(i + 1, COUNT - 1):
            dist_before = calclateTotalDistance(arg_order_vertex)
            next_order_vertex = nextVertex(arg_order_vertex, i, j)
            dist_after = calclateTotalDistance(next_order_vertex)
            exchange_cost = dist_after - dist_before
            if(exchange_cost < 0):
                arg_order_vertex = next_order_vertex
            else:
                prob = calcProbability(exchange_cost)
                if (random.random() <= prob):
                    arg_order_vertex = next_order_vertex
    return arg_order_vertex

# 頂点番号の入れ替え
def nextVertex(arg_order_vertex, i, j):
    arr = arg_order_vertex.copy()
    swap = arr[i]
    arr[i] = arr[j]
    arr[j] = swap
    return arr

# 温度による遷移確率
def calcProbability(exchange_cost):
    return math.exp(-exchange_cost  / temp)

# 総距離
def calclateTotalDistance(vertex):
    tmp_total_dist = 0
    for i in range(COUNT - 1):
        tmp_total_dist += dist_mat[vertex[i]][vertex[i+1]]
    tmp_total_dist += dist_mat[vertex[COUNT - 1]][vertex[0]]
    return tmp_total_dist

random.seed()
setupSA()
sa(DRAW_TIMES, DRAW_ITER)
print("ーーーーーーーーーーーーー")
print("最小距離 : "+ str(min_total_dist))
drawPath(min_order_vertex)

### 考察（要検証）


### 閃き方法 (要検証)

- 熱力学
- 鍛冶屋

### 生活の知恵　（要検証）



### 参考文献

- [焼きなまし法(Simulated Annealing)と2-Opt法による巡回セールスマン問題の解法](https://salad-bowl-of-knowledge.github.io/hp/processing/2017/12/03/simulated_annealing.html)


### 次

- [理論2層：近傍](%E7%90%86%E8%AB%962%E5%B1%A4%EF%BC%9A%E8%BF%91%E5%82%8D.ipynb)
- [理論2層：合成](%E7%90%86%E8%AB%962%E5%B1%A4%EF%BC%9A%E5%90%88%E6%88%90.ipynb)
- [理論2層：汎用](%E7%90%86%E8%AB%962%E5%B1%A4%EF%BC%9A%E6%B1%8E%E7%94%A8.ipynb)
- [理論2と人：評価](%E7%90%86%E8%AB%962%E5%B1%A4%E3%81%A8%E4%BA%BA%EF%BC%9A%E8%A9%95%E4%BE%A1.ipynb)
- [理論2と創造](%E7%90%86%E8%AB%962%E5%B1%A4%E3%81%A8%E5%89%B5%E9%80%A0.ipynb)
